In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
weekday=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data['weeknum'] = np.nan
test['weeknum'] = np.nan
for idx, day in enumerate(weekday):
    data['weeknum'] = np.where(data['Weekday'] == day, idx, data['weeknum'])
    test['weeknum'] = np.where(test['Weekday'] == day, idx, test['weeknum'])

In [4]:
train_test = pd.concat([data,test], axis=0)
train_test.shape

(1300700, 8)

In [5]:
train_test['Upc'] = train_test['Upc'].fillna(value = 9999)
train_test['FinelineNumber'] = train_test['FinelineNumber'].fillna(value = 9999)
train_test['DepartmentDescription'] = train_test['DepartmentDescription'].fillna(value = 'Nodata')

In [6]:
train_test['count'] = train_test['ScanCount']
train_test['count'][train_test['ScanCount']<0] = 0

/Users/doyoung/.pyenv/versions/anaconda3-5.0.0/envs/dodo/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
df = train_test.groupby(['VisitNumber', 'DepartmentDescription'],\
                       as_index = False)['count'].sum()
df.head()

,VisitNumber,DepartmentDescription,count
0,1,DAIRY,2
1,1,GROCERY DRY GOODS,1
2,1,SHOES,1
3,2,BAKERY,1
4,2,DSD GROCERY,1


In [8]:
df1 = df.groupby(['VisitNumber'], as_index=False)['count'].min()
df1.columns = ['VisitNumber', 'scancount_min']
df1.head()
df1.shape

(191348, 2)

In [9]:
df2 = df.groupby(['VisitNumber'], as_index=False)['count'].max()
df2.columns = ['VisitNumber', 'scancount_max']
df2.head()

,VisitNumber,scancount_max
0,1,2
1,2,2
2,3,1
3,4,1
4,5,0


In [10]:
df3 = df.groupby(['VisitNumber'], as_index=False)['count'].mean()
df3.columns = ['VisitNumber', 'scancount_mean']
df3.scancount_mean = round(df3.scancount_mean,2)
df3.head()

,VisitNumber,scancount_mean
0,1,1.33
1,2,1.33
2,3,1.00
3,4,1.00
4,5,0.00


In [11]:
df_to_be_added = pd.merge(df1,df2,how='outer',on="VisitNumber", )
df_to_be_added = pd.merge(df_to_be_added,df3,how='outer',on="VisitNumber", )
print(df_to_be_added.shape)
df_to_be_added.head()

(191348, 4)


,VisitNumber,scancount_min,scancount_max,scancount_mean
0,1,1,2,1.33
1,2,1,2,1.33
2,3,1,1,1.00
3,4,1,1,1.00
4,5,0,0,0.00


In [12]:
df_to_be_added['Max_Min'] = df_to_be_added['scancount_max'] - df_to_be_added['scancount_min']
df_to_be_added.head()

,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min
0,1,1,2,1.33,1
1,2,1,2,1.33,1
2,3,1,1,1.00,0
3,4,1,1,1.00,0
4,5,0,0,0.00,0


In [13]:
df = train_test[train_test['DepartmentDescription']=='Nodata']
df.head()

,DepartmentDescription,FinelineNumber,ScanCount,TripType,Upc,VisitNumber,Weekday,weeknum,count
25,Nodata,9999.0,1,26.0,9999.0,8,Friday,4.0,1
548,Nodata,9999.0,3,27.0,9999.0,259,Friday,4.0,3
549,Nodata,9999.0,1,27.0,9999.0,259,Friday,4.0,1
959,Nodata,9999.0,-1,999.0,9999.0,409,Friday,4.0,0
1116,Nodata,9999.0,1,39.0,9999.0,479,Friday,4.0,1


In [14]:
df = df.groupby(['VisitNumber'], as_index=False)['count'].count()
print(df.shape)
df.columns = ['VisitNumber', 'Num_of_Missing_department']
df.head()

(2313, 2)


,VisitNumber,Num_of_Missing_department
0,8,1
1,254,1
2,259,2
3,409,1
4,479,1


In [15]:
# df_to_be_added.drop(['Num_of_Missing_department','Num_of_Missing_fineline'], axis=1, inplace=True)
# df_to_be_added.head()

In [16]:
df_to_be_added = pd.merge(df_to_be_added,df,how='left',\
                          on='VisitNumber')
df_to_be_added.Num_of_Missing_department =\
df_to_be_added['Num_of_Missing_department'].fillna(0)
print(df_to_be_added.shape)
df_to_be_added.head()

(191348, 6)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department
0,1,1,2,1.33,1,0.0
1,2,1,2,1.33,1,0.0
2,3,1,1,1.00,0,0.0
3,4,1,1,1.00,0,0.0
4,5,0,0,0.00,0,0.0


In [17]:
df_to_be_added.loc[258,'Num_of_Missing_department']

2.0

In [18]:
df_100 = train_test[train_test['FinelineNumber']==9999]
df_100.head()

,DepartmentDescription,FinelineNumber,ScanCount,TripType,Upc,VisitNumber,Weekday,weeknum,count
25,Nodata,9999.0,1,26.0,9999.0,8,Friday,4.0,1
548,Nodata,9999.0,3,27.0,9999.0,259,Friday,4.0,3
549,Nodata,9999.0,1,27.0,9999.0,259,Friday,4.0,1
959,Nodata,9999.0,-1,999.0,9999.0,409,Friday,4.0,0
1116,Nodata,9999.0,1,39.0,9999.0,479,Friday,4.0,1


In [19]:
len(df.index) == len(df_100.index)

False

In [20]:
df_100 = df_100.groupby(['VisitNumber'], as_index=False)['count'].count()
print(df_100.shape)
df_100.columns = ['VisitNumber', 'Num_of_Missing_fineline']
df_100.head()

(5461, 2)


,VisitNumber,Num_of_Missing_fineline
0,8,1
1,254,1
2,259,2
3,409,1
4,479,1


In [21]:
sum(df.Num_of_Missing_department) -sum(df_100.Num_of_Missing_fineline) 

-5427

In [22]:
df_to_be_added = pd.merge(df_to_be_added,df_100,how='left',\
                          on='VisitNumber')
df_to_be_added.Num_of_Missing_fineline =\
df_to_be_added['Num_of_Missing_fineline'].fillna(0)
print(df_to_be_added.shape)
df_to_be_added.head(10)

(191348, 7)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline
0,1,1,2,1.33,1,0.0,0.0
1,2,1,2,1.33,1,0.0,0.0
2,3,1,1,1.00,0,0.0,0.0
3,4,1,1,1.00,0,0.0,0.0
4,5,0,0,0.00,0,0.0,0.0
5,6,1,1,1.00,0,0.0,0.0
6,7,1,1,1.00,0,0.0,0.0
7,8,1,20,4.29,19,1.0,1.0
8,9,1,2,1.50,1,0.0,0.0
9,10,1,2,1.50,1,0.0,0.0


In [23]:
sum(df_to_be_added.Num_of_Missing_department - df_to_be_added.Num_of_Missing_fineline)

-5427.0

In [24]:
train_test.head()

,DepartmentDescription,FinelineNumber,ScanCount,TripType,Upc,VisitNumber,Weekday,weeknum,count
0,FINANCIAL SERVICES,1000.0,-1,999.0,6.811315e+10,5,Friday,4.0,0
1,SHOES,8931.0,1,30.0,6.053882e+10,7,Friday,4.0,1
2,PERSONAL CARE,4504.0,1,30.0,7.410811e+09,7,Friday,4.0,1
3,PAINT AND ACCESSORIES,3565.0,2,26.0,2.238404e+09,8,Friday,4.0,2
4,PAINT AND ACCESSORIES,1017.0,2,26.0,2.006614e+09,8,Friday,4.0,2


In [25]:
train_test['Return'] = np.where(train_test['ScanCount'] < 0, 1, 0)
train_test.head()

,DepartmentDescription,FinelineNumber,ScanCount,TripType,Upc,VisitNumber,Weekday,weeknum,count,Return
0,FINANCIAL SERVICES,1000.0,-1,999.0,6.811315e+10,5,Friday,4.0,0,1
1,SHOES,8931.0,1,30.0,6.053882e+10,7,Friday,4.0,1,0
2,PERSONAL CARE,4504.0,1,30.0,7.410811e+09,7,Friday,4.0,1,0
3,PAINT AND ACCESSORIES,3565.0,2,26.0,2.238404e+09,8,Friday,4.0,2,0
4,PAINT AND ACCESSORIES,1017.0,2,26.0,2.006614e+09,8,Friday,4.0,2,0


In [26]:
df = train_test[train_test.Return==1]
df = df.groupby('VisitNumber', as_index=False)['count'].count()
df['boolean_return'] =1 
df.drop('count', axis=1, inplace=True)
df.head()

,VisitNumber,boolean_return
0,3,1
1,5,1
2,6,1
3,8,1
4,52,1


In [27]:
df_to_be_added = pd.merge(df_to_be_added, df, how='left',\
                          on='VisitNumber',)
df_to_be_added.boolean_return =\
df_to_be_added['boolean_return'].fillna(0)
print(df_to_be_added.shape)
df_to_be_added.head(10)

(191348, 8)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return
0,1,1,2,1.33,1,0.0,0.0,0.0
1,2,1,2,1.33,1,0.0,0.0,0.0
2,3,1,1,1.00,0,0.0,0.0,1.0
3,4,1,1,1.00,0,0.0,0.0,0.0
4,5,0,0,0.00,0,0.0,0.0,1.0
5,6,1,1,1.00,0,0.0,0.0,1.0
6,7,1,1,1.00,0,0.0,0.0,0.0
7,8,1,20,4.29,19,1.0,1.0,1.0
8,9,1,2,1.50,1,0.0,0.0,0.0
9,10,1,2,1.50,1,0.0,0.0,0.0


In [28]:
train_test[train_test.VisitNumber == 3]

,DepartmentDescription,FinelineNumber,ScanCount,TripType,Upc,VisitNumber,Weekday,weeknum,count,Return
8,PERSONAL CARE,4504.0,1,NaN,7.410811e+09,3,Friday,4.0,1,0
9,PERSONAL CARE,4504.0,-1,NaN,7.410811e+09,3,Friday,4.0,0,1


In [29]:
df = train_test.groupby(['VisitNumber',  'FinelineNumber'], as_index=False)['count'].count()
df = train_test.groupby(['VisitNumber'], as_index=False)['count'].count()
df.columns = ['VisitNumber', 'Num_Fineline_bought']
df.head()

,VisitNumber,Num_Fineline_bought
0,1,4
1,2,4
2,3,2
3,4,1
4,5,1


In [30]:
df_to_be_added = pd.merge(df_to_be_added, df, how='left',\
                          on='VisitNumber',)
df_to_be_added.Num_Fineline_bought =\
df_to_be_added['Num_Fineline_bought'].fillna(0)
print(df_to_be_added.shape)
df_to_be_added.head(10)

(191348, 9)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return,Num_Fineline_bought
0,1,1,2,1.33,1,0.0,0.0,0.0,4
1,2,1,2,1.33,1,0.0,0.0,0.0,4
2,3,1,1,1.00,0,0.0,0.0,1.0,2
3,4,1,1,1.00,0,0.0,0.0,0.0,1
4,5,0,0,0.00,0,0.0,0.0,1.0,1
5,6,1,1,1.00,0,0.0,0.0,1.0,2
6,7,1,1,1.00,0,0.0,0.0,0.0,2
7,8,1,20,4.29,19,1.0,1.0,1.0,23
8,9,1,2,1.50,1,0.0,0.0,0.0,3
9,10,1,2,1.50,1,0.0,0.0,0.0,3


In [31]:
df = train_test.groupby(['VisitNumber',  'Upc'], as_index=False)['count'].count()
df = df.groupby(['VisitNumber'], as_index=False)['count'].count()
df.rename(columns={'count': 'Num_Upc_bought'}, inplace=True) 
df.head()

,VisitNumber,Num_Upc_bought
0,1,4
1,2,4
2,3,1
3,4,1
4,5,1


In [32]:
df_to_be_added = pd.merge(df_to_be_added, df, how='left',\
                          on='VisitNumber',)
df_to_be_added.Num_Upc_bought =\
df_to_be_added['Num_Upc_bought'].fillna(0)
print(df_to_be_added.shape)
df_to_be_added.head(10)

(191348, 10)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return,Num_Fineline_bought,Num_Upc_bought
0,1,1,2,1.33,1,0.0,0.0,0.0,4,4
1,2,1,2,1.33,1,0.0,0.0,0.0,4,4
2,3,1,1,1.00,0,0.0,0.0,1.0,2,1
3,4,1,1,1.00,0,0.0,0.0,0.0,1,1
4,5,0,0,0.00,0,0.0,0.0,1.0,1,1
5,6,1,1,1.00,0,0.0,0.0,1.0,2,1
6,7,1,1,1.00,0,0.0,0.0,0.0,2,2
7,8,1,20,4.29,19,1.0,1.0,1.0,23,21
8,9,1,2,1.50,1,0.0,0.0,0.0,3,3
9,10,1,2,1.50,1,0.0,0.0,0.0,3,3


In [33]:
df = train_test.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['count'].count()
df = df.groupby(['VisitNumber'], as_index=False)['count'].count()
df.rename(columns={'count': 'Num_Department_bought'}, inplace=True)
df.head()

,VisitNumber,Num_Department_bought
0,1,3
1,2,3
2,3,1
3,4,1
4,5,1


In [34]:
df_to_be_added = pd.merge(df_to_be_added, df, how='left',\
                          on='VisitNumber',)
df_to_be_added.Num_Department_bought =\
df_to_be_added['Num_Department_bought'].fillna(0)
print(df_to_be_added.shape)
df_to_be_added.head(10)

(191348, 11)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return,Num_Fineline_bought,Num_Upc_bought,Num_Department_bought
0,1,1,2,1.33,1,0.0,0.0,0.0,4,4,3
1,2,1,2,1.33,1,0.0,0.0,0.0,4,4,3
2,3,1,1,1.00,0,0.0,0.0,1.0,2,1,1
3,4,1,1,1.00,0,0.0,0.0,0.0,1,1,1
4,5,0,0,0.00,0,0.0,0.0,1.0,1,1,1
5,6,1,1,1.00,0,0.0,0.0,1.0,2,1,1
6,7,1,1,1.00,0,0.0,0.0,0.0,2,2,2
7,8,1,20,4.29,19,1.0,1.0,1.0,23,21,7
8,9,1,2,1.50,1,0.0,0.0,0.0,3,3,2
9,10,1,2,1.50,1,0.0,0.0,0.0,3,3,2


In [35]:
df_to_be_added['Pct_F_D'] = df_to_be_added['Num_Fineline_bought']/df_to_be_added['Num_Department_bought']
df_to_be_added['Pct_U_D'] = df_to_be_added['Num_Upc_bought']/df_to_be_added['Num_Department_bought'] 
df_to_be_added['mean_by_min'] = df_to_be_added['scancount_mean']/df_to_be_added['scancount_min']
df_to_be_added['mean_by_min'][df_to_be_added['scancount_min']==0] = 0
df_to_be_added['max_by_mean'] = df_to_be_added['scancount_max']/df_to_be_added['scancount_mean']
df_to_be_added['max_by_mean'][df_to_be_added['scancount_mean']==0] = 0    

/Users/doyoung/.pyenv/versions/anaconda3-5.0.0/envs/dodo/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/doyoung/.pyenv/versions/anaconda3-5.0.0/envs/dodo/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
print(df_to_be_added.shape)
df_to_be_added.head(10)

(191348, 15)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return,Num_Fineline_bought,Num_Upc_bought,Num_Department_bought,Pct_F_D,Pct_U_D,mean_by_min,max_by_mean
0,1,1,2,1.33,1,0.0,0.0,0.0,4,4,3,1.333333,1.333333,1.33,1.503759
1,2,1,2,1.33,1,0.0,0.0,0.0,4,4,3,1.333333,1.333333,1.33,1.503759
2,3,1,1,1.00,0,0.0,0.0,1.0,2,1,1,2.000000,1.000000,1.00,1.000000
3,4,1,1,1.00,0,0.0,0.0,0.0,1,1,1,1.000000,1.000000,1.00,1.000000
4,5,0,0,0.00,0,0.0,0.0,1.0,1,1,1,1.000000,1.000000,0.00,0.000000
5,6,1,1,1.00,0,0.0,0.0,1.0,2,1,1,2.000000,1.000000,1.00,1.000000
6,7,1,1,1.00,0,0.0,0.0,0.0,2,2,2,1.000000,1.000000,1.00,1.000000
7,8,1,20,4.29,19,1.0,1.0,1.0,23,21,7,3.285714,3.000000,4.29,4.662005
8,9,1,2,1.50,1,0.0,0.0,0.0,3,3,2,1.500000,1.500000,1.50,1.333333
9,10,1,2,1.50,1,0.0,0.0,0.0,3,3,2,1.500000,1.500000,1.50,1.333333


In [37]:
department_list = np.unique(train_test.DepartmentDescription)

In [38]:
len(department_list)

69

In [39]:
new_data=train_test[['VisitNumber', 'Weekday']].drop_duplicates()
for i in range(len(department_list)):
    df=train_test[(train_test['DepartmentDescription']==department_list[i]) & (train_test['ScanCount']<0)]
    df = df.groupby(['VisitNumber'], as_index=False)['ScanCount'].sum()
    df['ScanCount'] = df['ScanCount']*(-1)
    df.rename(columns={'ScanCount': 'D_Neg_pro_%s' % (i)}, inplace=True)
    new_data = new_data.merge(df, how='left', on=['VisitNumber'], copy=True)
    new_data['D_Neg_pro_%s' % (i)].fillna(value=0, inplace=True)
new_data.drop('Weekday', axis=1, inplace=True)   
new_data

,VisitNumber,D_Neg_pro_0,D_Neg_pro_1,D_Neg_pro_2,D_Neg_pro_3,D_Neg_pro_4,D_Neg_pro_5,D_Neg_pro_6,D_Neg_pro_7,D_Neg_pro_8,...,D_Neg_pro_59,D_Neg_pro_60,D_Neg_pro_61,D_Neg_pro_62,D_Neg_pro_63,D_Neg_pro_64,D_Neg_pro_65,D_Neg_pro_66,D_Neg_pro_67,D_Neg_pro_68
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
new_data1=train_test[['VisitNumber', 'Weekday']].drop_duplicates()  
for i in range(len(department_list)):
    df=train_test[(train_test['DepartmentDescription']==department_list[i]) & (train_test['ScanCount']>1)]
    df = df.groupby(['VisitNumber'], as_index=False)['ScanCount'].count()
    df.rename(columns={'ScanCount': 'D_Num_multi_pro_%s' % (i)}, inplace=True)
    new_data1 = new_data1.merge(df, how='left', on=['VisitNumber'], copy=True)
    new_data1['D_Num_multi_pro_%s' % (i)].fillna(value=0, inplace=True)
new_data1.drop('Weekday', axis=1, inplace=True)   
new_data1  


,VisitNumber,D_Num_multi_pro_0,D_Num_multi_pro_1,D_Num_multi_pro_2,D_Num_multi_pro_3,D_Num_multi_pro_4,D_Num_multi_pro_5,D_Num_multi_pro_6,D_Num_multi_pro_7,D_Num_multi_pro_8,...,D_Num_multi_pro_59,D_Num_multi_pro_60,D_Num_multi_pro_61,D_Num_multi_pro_62,D_Num_multi_pro_63,D_Num_multi_pro_64,D_Num_multi_pro_65,D_Num_multi_pro_66,D_Num_multi_pro_67,D_Num_multi_pro_68
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
print(df_to_be_added.shape)
df_to_be_added.head()

(191348, 153)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return,Num_Fineline_bought,Num_Upc_bought,...,D_Num_multi_pro_59,D_Num_multi_pro_60,D_Num_multi_pro_61,D_Num_multi_pro_62,D_Num_multi_pro_63,D_Num_multi_pro_64,D_Num_multi_pro_65,D_Num_multi_pro_66,D_Num_multi_pro_67,D_Num_multi_pro_68
0,1,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,1,1.00,0,0.0,0.0,1.0,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,1,1.00,0,0.0,0.0,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0,0.00,0,0.0,0.0,1.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df_to_be_added = pd.merge(df_to_be_added, new_data, how='left',\
                          on='VisitNumber',)
print(df_to_be_added.shape)
df_to_be_added.head(10)

(191348, 222)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return,Num_Fineline_bought,Num_Upc_bought,...,D_Neg_pro_59_y,D_Neg_pro_60_y,D_Neg_pro_61_y,D_Neg_pro_62_y,D_Neg_pro_63_y,D_Neg_pro_64_y,D_Neg_pro_65_y,D_Neg_pro_66_y,D_Neg_pro_67_y,D_Neg_pro_68_y
0,1,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,1,1.00,0,0.0,0.0,1.0,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,1,1.00,0,0.0,0.0,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0,0.00,0,0.0,0.0,1.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,1,1,1.00,0,0.0,0.0,1.0,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,1,1,1.00,0,0.0,0.0,0.0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,1,20,4.29,19,1.0,1.0,1.0,23,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,1,2,1.50,1,0.0,0.0,0.0,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,1,2,1.50,1,0.0,0.0,0.0,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
df_to_be_added = pd.merge(df_to_be_added, new_data1, how='left',\
                          on='VisitNumber',)
print(df_to_be_added.shape)
df_to_be_added.head(10)

(191348, 291)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return,Num_Fineline_bought,Num_Upc_bought,...,D_Num_multi_pro_59_y,D_Num_multi_pro_60_y,D_Num_multi_pro_61_y,D_Num_multi_pro_62_y,D_Num_multi_pro_63_y,D_Num_multi_pro_64_y,D_Num_multi_pro_65_y,D_Num_multi_pro_66_y,D_Num_multi_pro_67_y,D_Num_multi_pro_68_y
0,1,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,1,1.00,0,0.0,0.0,1.0,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,1,1.00,0,0.0,0.0,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0,0.00,0,0.0,0.0,1.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,1,1,1.00,0,0.0,0.0,1.0,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,1,1,1.00,0,0.0,0.0,0.0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,1,20,4.29,19,1.0,1.0,1.0,23,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,1,2,1.50,1,0.0,0.0,0.0,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,1,2,1.50,1,0.0,0.0,0.0,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
train_test.head()

,DepartmentDescription,FinelineNumber,ScanCount,TripType,Upc,VisitNumber,Weekday,weeknum,count,Return
0,FINANCIAL SERVICES,1000.0,-1,999.0,6.811315e+10,5,Friday,4.0,0,1
1,SHOES,8931.0,1,30.0,6.053882e+10,7,Friday,4.0,1,0
2,PERSONAL CARE,4504.0,1,30.0,7.410811e+09,7,Friday,4.0,1,0
3,PAINT AND ACCESSORIES,3565.0,2,26.0,2.238404e+09,8,Friday,4.0,2,0
4,PAINT AND ACCESSORIES,1017.0,2,26.0,2.006614e+09,8,Friday,4.0,2,0


In [48]:
train_test_depart_dummied = train_test.copy()

In [49]:
dum = pd.get_dummies(train_test['DepartmentDescription'])
train_test_depart_dummied[dum.columns] = dum
train_test_depart_dummied.head()

,DepartmentDescription,FinelineNumber,ScanCount,TripType,Upc,VisitNumber,Weekday,weeknum,count,Return,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,FINANCIAL SERVICES,1000.0,-1,999.0,6.811315e+10,5,Friday,4.0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,SHOES,8931.0,1,30.0,6.053882e+10,7,Friday,4.0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,PERSONAL CARE,4504.0,1,30.0,7.410811e+09,7,Friday,4.0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,PAINT AND ACCESSORIES,3565.0,2,26.0,2.238404e+09,8,Friday,4.0,2,0,...,0,0,0,0,0,0,0,0,0,0
4,PAINT AND ACCESSORIES,1017.0,2,26.0,2.006614e+09,8,Friday,4.0,2,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
grouped_data = train_test_depart_dummied.groupby("VisitNumber")
grouped_data = grouped_data.agg({'1-HR PHOTO': np.sum,'ACCESSORIES': np.sum,'AUTOMOTIVE': np.sum,'BAKERY': np.sum,'BATH AND SHOWER': np.sum,'BEAUTY': np.sum,'BEDDING': np.sum,'BOOKS AND MAGAZINES': np.sum,'BOYS WEAR': np.sum,'BRAS & SHAPEWEAR': np.sum,'CAMERAS AND SUPPLIES': np.sum,'CANDY, TOBACCO, COOKIES': np.sum,'CELEBRATION': np.sum,'COMM BREAD': np.sum,'CONCEPT STORES': np.sum,'COOK AND DINE': np.sum,'DAIRY': np.sum,'DSD GROCERY': np.sum,'ELECTRONICS': np.sum,'FABRICS AND CRAFTS': np.sum,'FINANCIAL SERVICES': np.sum,'FROZEN FOODS': np.sum,'FURNITURE': np.sum,'GIRLS WEAR, 4-6X  AND 7-14': np.sum,'GROCERY DRY GOODS': np.sum,'HARDWARE': np.sum,'HEALTH AND BEAUTY AIDS': np.sum,'HOME DECOR': np.sum,'HOME MANAGEMENT': np.sum,'HORTICULTURE AND ACCESS': np.sum,'HOUSEHOLD CHEMICALS/SUPP': np.sum,'HOUSEHOLD PAPER GOODS': np.sum,'IMPULSE MERCHANDISE': np.sum,'INFANT APPAREL': np.sum,'INFANT CONSUMABLE HARDLINES': np.sum,'JEWELRY AND SUNGLASSES': np.sum,'LADIES SOCKS': np.sum,'LADIESWEAR': np.sum,'LARGE HOUSEHOLD GOODS': np.sum,'LAWN AND GARDEN': np.sum,'LIQUOR,WINE,BEER': np.sum,'MEAT - FRESH & FROZEN': np.sum,'MEDIA AND GAMING': np.sum,'MENS WEAR': np.sum,'MENSWEAR': np.sum,'Nodata': np.sum,'OFFICE SUPPLIES': np.sum,'OPTICAL - FRAMES': np.sum,'OPTICAL - LENSES': np.sum,'OTHER DEPARTMENTS': np.sum,'PAINT AND ACCESSORIES': np.sum,'PERSONAL CARE': np.sum,'PETS AND SUPPLIES': np.sum,'PHARMACY OTC': np.sum,'PHARMACY RX': np.sum,'PLAYERS AND ELECTRONICS': np.sum,'PLUS AND MATERNITY': np.sum,'PRE PACKED DELI': np.sum,'PRODUCE': np.sum,'SEAFOOD': np.sum,'SEASONAL': np.sum,'SERVICE DELI': np.sum,'SHEER HOSIERY': np.sum,'SHOES': np.sum,'SLEEPWEAR/FOUNDATIONS': np.sum,'SPORTING GOODS': np.sum,'SWIMWEAR/OUTERWEAR': np.sum,'TOYS': np.sum,'WIRELESS': np.sum})
grouped_data.head()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
importance = np.sum(grouped_data, axis=0).values
idx = np.argsort(np.sum(grouped_data, axis=0))[::-1]

In [52]:
department_66pct_list = []
for f in range(69):
    department_66pct_list.append(grouped_data.columns[idx[f]]) 
    print("%d. %s %f" % (f + 1, grouped_data.columns[idx[f]], importance[idx[f]]))


1. GROCERY DRY GOODS 142737.000000
2. DSD GROCERY 137192.000000
3. PRODUCE 102043.000000
4. DAIRY 88644.000000
5. PERSONAL CARE 83576.000000
6. IMPULSE MERCHANDISE 57153.000000
7. HOUSEHOLD CHEMICALS/SUPP 49482.000000
8. PHARMACY OTC 46599.000000
9. FROZEN FOODS 42991.000000
10. HOUSEHOLD PAPER GOODS 31916.000000
11. COMM BREAD 30932.000000
12. BEAUTY 30088.000000
13. MENS WEAR 24421.000000
14. FINANCIAL SERVICES 21580.000000
15. INFANT CONSUMABLE HARDLINES 21316.000000
16. SERVICE DELI 20176.000000
17. CANDY, TOBACCO, COOKIES 20118.000000
18. PETS AND SUPPLIES 19455.000000
19. MEAT - FRESH & FROZEN 19413.000000
20. PRE PACKED DELI 18328.000000
21. LADIESWEAR 17740.000000
22. CELEBRATION 17333.000000
23. COOK AND DINE 15388.000000
24. BAKERY 14333.000000
25. OFFICE SUPPLIES 13371.000000
26. HOME MANAGEMENT 12646.000000
27. INFANT APPAREL 12610.000000
28. SHOES 12313.000000
29. TOYS 12242.000000
30. AUTOMOTIVE 10873.000000
31. LAWN AND GARDEN 10790.000000
32. FABRICS AND CRAFTS 10221.00

In [53]:
department_66pct_list =\
department_66pct_list[:int(len(department_66pct_list)*5.5/10)]
len(department_66pct_list)

37

In [54]:
main_depart = grouped_data.loc[:,department_66pct_list]

In [55]:
for i in range(36):
        for j in range(i+1, len(department_66pct_list)):
            main_depart['D_%s_%s' % (department_66pct_list[i], department_66pct_list[j])] = \
            main_depart[(department_66pct_list[i])]*main_depart[(department_66pct_list[j])]
            

In [56]:
main_depart

,GROCERY DRY GOODS,DSD GROCERY,PRODUCE,DAIRY,PERSONAL CARE,IMPULSE MERCHANDISE,HOUSEHOLD CHEMICALS/SUPP,PHARMACY OTC,FROZEN FOODS,HOUSEHOLD PAPER GOODS,...,D_SPORTING GOODS_HARDWARE,D_SPORTING GOODS_BATH AND SHOWER,"D_SPORTING GOODS_GIRLS WEAR, 4-6X AND 7-14",D_SPORTING GOODS_HOME DECOR,D_HARDWARE_BATH AND SHOWER,"D_HARDWARE_GIRLS WEAR, 4-6X AND 7-14",D_HARDWARE_HOME DECOR,"D_BATH AND SHOWER_GIRLS WEAR, 4-6X AND 7-14",D_BATH AND SHOWER_HOME DECOR,"D_GIRLS WEAR, 4-6X AND 7-14_HOME DECOR"
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
main_depart.shape

(191348, 703)

In [58]:
main_depart.drop(department_66pct_list, axis =1, inplace = True)
main_depart.shape

(191348, 666)

In [59]:
df_to_be_added.head()

,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return,Num_Fineline_bought,Num_Upc_bought,...,D_Num_multi_pro_59_y,D_Num_multi_pro_60_y,D_Num_multi_pro_61_y,D_Num_multi_pro_62_y,D_Num_multi_pro_63_y,D_Num_multi_pro_64_y,D_Num_multi_pro_65_y,D_Num_multi_pro_66_y,D_Num_multi_pro_67_y,D_Num_multi_pro_68_y
0,1,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,1,1.00,0,0.0,0.0,1.0,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,1,1.00,0,0.0,0.0,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0,0.00,0,0.0,0.0,1.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
main_depart.head()

,D_GROCERY DRY GOODS_DSD GROCERY,D_GROCERY DRY GOODS_PRODUCE,D_GROCERY DRY GOODS_DAIRY,D_GROCERY DRY GOODS_PERSONAL CARE,D_GROCERY DRY GOODS_IMPULSE MERCHANDISE,D_GROCERY DRY GOODS_HOUSEHOLD CHEMICALS/SUPP,D_GROCERY DRY GOODS_PHARMACY OTC,D_GROCERY DRY GOODS_FROZEN FOODS,D_GROCERY DRY GOODS_HOUSEHOLD PAPER GOODS,D_GROCERY DRY GOODS_COMM BREAD,...,D_SPORTING GOODS_HARDWARE,D_SPORTING GOODS_BATH AND SHOWER,"D_SPORTING GOODS_GIRLS WEAR, 4-6X AND 7-14",D_SPORTING GOODS_HOME DECOR,D_HARDWARE_BATH AND SHOWER,"D_HARDWARE_GIRLS WEAR, 4-6X AND 7-14",D_HARDWARE_HOME DECOR,"D_BATH AND SHOWER_GIRLS WEAR, 4-6X AND 7-14",D_BATH AND SHOWER_HOME DECOR,"D_GIRLS WEAR, 4-6X AND 7-14_HOME DECOR"
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
# main_depart.drop('VisitNumber', axis=1,inplace =True)
main_depart.reset_index(level=['VisitNumber'], inplace=True)
main_depart.head()

,VisitNumber,D_GROCERY DRY GOODS_DSD GROCERY,D_GROCERY DRY GOODS_PRODUCE,D_GROCERY DRY GOODS_DAIRY,D_GROCERY DRY GOODS_PERSONAL CARE,D_GROCERY DRY GOODS_IMPULSE MERCHANDISE,D_GROCERY DRY GOODS_HOUSEHOLD CHEMICALS/SUPP,D_GROCERY DRY GOODS_PHARMACY OTC,D_GROCERY DRY GOODS_FROZEN FOODS,D_GROCERY DRY GOODS_HOUSEHOLD PAPER GOODS,...,D_SPORTING GOODS_HARDWARE,D_SPORTING GOODS_BATH AND SHOWER,"D_SPORTING GOODS_GIRLS WEAR, 4-6X AND 7-14",D_SPORTING GOODS_HOME DECOR,D_HARDWARE_BATH AND SHOWER,"D_HARDWARE_GIRLS WEAR, 4-6X AND 7-14",D_HARDWARE_HOME DECOR,"D_BATH AND SHOWER_GIRLS WEAR, 4-6X AND 7-14",D_BATH AND SHOWER_HOME DECOR,"D_GIRLS WEAR, 4-6X AND 7-14_HOME DECOR"
0,1,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
df_to_be_added = pd.merge(df_to_be_added, main_depart, how='left',\
                          on='VisitNumber',)
print(df_to_be_added.shape)
df_to_be_added.head(10)

(191348, 957)


,VisitNumber,scancount_min,scancount_max,scancount_mean,Max_Min,Num_of_Missing_department,Num_of_Missing_fineline,boolean_return,Num_Fineline_bought,Num_Upc_bought,...,D_SPORTING GOODS_HARDWARE,D_SPORTING GOODS_BATH AND SHOWER,"D_SPORTING GOODS_GIRLS WEAR, 4-6X AND 7-14",D_SPORTING GOODS_HOME DECOR,D_HARDWARE_BATH AND SHOWER,"D_HARDWARE_GIRLS WEAR, 4-6X AND 7-14",D_HARDWARE_HOME DECOR,"D_BATH AND SHOWER_GIRLS WEAR, 4-6X AND 7-14",D_BATH AND SHOWER_HOME DECOR,"D_GIRLS WEAR, 4-6X AND 7-14_HOME DECOR"
0,1,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0,0,0,0,0,0,0,0,0,0
1,2,1,2,1.33,1,0.0,0.0,0.0,4,4,...,0,0,0,0,0,0,0,0,0,0
2,3,1,1,1.00,0,0.0,0.0,1.0,2,1,...,0,0,0,0,0,0,0,0,0,0
3,4,1,1,1.00,0,0.0,0.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0.00,0,0.0,0.0,1.0,1,1,...,0,0,0,0,0,0,0,0,0,0
5,6,1,1,1.00,0,0.0,0.0,1.0,2,1,...,0,0,0,0,0,0,0,0,0,0
6,7,1,1,1.00,0,0.0,0.0,0.0,2,2,...,0,0,0,0,0,0,0,0,0,0
7,8,1,20,4.29,19,1.0,1.0,1.0,23,21,...,0,0,0,0,0,0,0,0,0,0
8,9,1,2,1.50,1,0.0,0.0,0.0,3,3,...,0,0,0,0,0,0,0,0,0,0
9,10,1,2,1.50,1,0.0,0.0,0.0,3,3,...,0,0,0,0,0,0,0,0,0,0


In [64]:
df_to_be_added.to_csv('df_to_be_added_be_added.csv',index=False)

## finelineNumber PCA


In [ ]:
# grouped_fineline = pd.read_csv('grouped_fineline.csv')

In [190]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2800)
pca.fit(grouped_fineline)

PCA(copy=True, iterated_power='auto', n_components=2800, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [191]:
grouped_fineline_pca = pca.transform(grouped_fineline)

In [194]:
grouped_fineline_pca.shape

(191348, 2800)

In [196]:
grouped_fineline_pca = pd.DataFrame(grouped_fineline_pca)

In [197]:
grouped_fineline_pca.to_csv('grouped_fineline_pca.csv', index=False)